In [ ]:
# !pip install biopython
# !pip install nglview
# !conda install -c conda-forge nglview

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### The CASP12 dataset is just a huge (12gb) ASCII text file with each protein's data in sequence. Open the `test` file if you want to see the structure as it is only a few mb. Here is a parser that loads it into an initial df.

In [3]:
import pandas as pd

def parse_protein_data(file_path, n):
    # Initialize lists to store data
    ids = []
    sequences = []
    evolutionary = []
    tertiary = []
    masks = []
    
    # Initialize counters and temporary storage for current protein data
    current_protein = {}
    count = 0
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('[ID]'):
                # Save previous protein data if exists
                if current_protein:
                    ids.append(current_protein.get('ID', ''))
                    sequences.append(current_protein.get('PRIMARY', ''))
                    evolutionary.append(current_protein.get('EVOLUTIONARY', []))
                    tertiary.append(current_protein.get('TERTIARY', []))
                    masks.append(current_protein.get('MASK', ''))
                    count += 1
                    # Break the loop if the required number of proteins have been parsed
                    if count >= n:
                        break
                    current_protein = {}
            
            # Identify the section and append the data to the current protein
            if line.startswith('['):
                key = line[1:line.find(']')]
                current_protein[key] = []
            elif current_protein:
                # Split TSV data into lists of floats or keep as strings depending on the section
                if key in ['EVOLUTIONARY', 'TERTIARY']:
                    # Split the line by spaces, convert each item to float
                    current_protein[key].append([float(x) for x in line.split()])
                else:
                    current_protein[key].append(line)
                
    # Check if last processed protein needs to be added
    if count < n and current_protein:
        ids.append(current_protein.get('ID', ''))
        sequences.append(current_protein.get('PRIMARY', ''))
        evolutionary.append(current_protein.get('EVOLUTIONARY', []))
        tertiary.append(current_protein.get('TERTIARY', []))
        masks.append(current_protein.get('MASK', ''))
    
    # Create DataFrame
    df = pd.DataFrame({
        'ID': ids,
        'Sequence': sequences,
        'Evolutionary': evolutionary,
        'Tertiary': tertiary,
        'Mask': masks
    })
    
    return df

In [4]:
# Usage example
file_path = './data/validation'
number_of_proteins = 300  # Set how many proteins you want to load
protein_df = parse_protein_data(file_path, number_of_proteins)
protein_df.head(2)

,ID,Sequence,Evolutionary,Tertiary,Mask
0,[90#2WXZ_2_C],[DRVYIHPFHLLYYSKSTCAQLENPSVETLPEPTFEPVPIQAKTSP...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[-----+++++++++++++++-------++++++++++++++++++...
1,[30#3U88_2_M],[SRWRFPARPGTGRRGLGGAPRQRVPALLRVGPGFDAALQVSAAIG...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.785860237024...","[[0.0, 0.0, 0.0, -6941.7, -7005.4, -6965.1, -6...",[-++++++++++++++++++++++++++++++++++++++++++++...


### We need to do some further extraction to get the right format:

In [5]:
def process_dataframe(df):
    # Convert ID, Sequence, and Mask to string directly
    df['ID'] = df['ID'].apply(lambda x: ''.join(x))
    df['Sequence'] = df['Sequence'].apply(lambda x: ''.join(x))
    df['Mask'] = df['Mask'].apply(lambda x: ''.join(x).replace('+', '1').replace('-', '0'))

    # Initialize columns for coordinates and amino acids
    coord_columns = ['x', 'y', 'z']
    aa_columns = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','Info']

    # Expand tertiary coordinates
    for i, col in enumerate(coord_columns):
        df[col] = df['Tertiary'].apply(lambda x: x[i] if len(x) > i else None)

    # Expand evolutionary data into separate columns for each amino acid position
    for i, col in enumerate(aa_columns):
        df[col] = df['Evolutionary'].apply(lambda x: x[i] if len(x) > i else None)

    # Drop the original Tertiary and Evolutionary columns
    df.drop(['Tertiary', 'Evolutionary'], axis=1, inplace=True)

    return df

In [6]:
# Assume 'df' is your DataFrame loaded with data
# Example usage:
processed_df = process_dataframe(protein_df)
processed_df.head(2)

,ID,Sequence,Mask,x,y,z,A,C,D,E,...,N,P,Q,R,S,T,V,W,Y,Info
0,90#2WXZ_2_C,DRVYIHPFHLLYYSKSTCAQLENPSVETLPEPTFEPVPIQAKTSPV...,0000011111111111111100000001111111111111111111...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.237...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.040...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021...","[0.0, 0.0, 0.9633596392333709, 0.0, 0.48795669...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.9787755102040815, 0.0, 0.0, ...","[0.9999999988691709, 0.9999999988691709, 0.939..."
1,30#3U88_2_M,SRWRFPARPGTGRRGLGGAPRQRVPALLRVGPGFDAALQVSAAIGT...,0111111111111111111111111111111111111111111111...,"[0.0, 0.0, 0.0, -6941.7, -7005.4, -6965.1, -68...","[0.0, 0.0, 0.0, 5551.7, 5538.0, 5409.6, 5380.7...","[0.0, 0.0, 0.0, 2541.6, 2410.3, 2334.6, 2325.8...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7858602370249...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.01074413...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.76283326...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2141397629750...","[0.0, 0.0, 0.06993736951983298, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.9300626304801669, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9999999988691709, 0.9999999988691709, 0.915..."


## Data description:
- ID is the protein name
- Sequence is the amino acid sequence using the single letter code
- Mask (same length as sequence) is a binary representation of how importatnt each amino acid is to the structure (that is, amino acids marked `0` can be changed without affecting the structure much, whereas amino acids marked `1` would significantly alter structure/function).
- XYZ are the coordinates of each atom (note there are 3 atoms per amino acid, Nitrogen, alpha-Carbon, and Carbon, so there are 3 times as many coordinates as amino acids in the sequence)
- The 20 columns A-Y are normalized PSSM information for each amino acid. This is essentially a sparse matrix where there is a row for each position in the protein sequence, and a column for each of the 20 amino acids.
- Info contains the information content for that residue (unclear what this actually is)
## Note:
Proteins are chains of amino acids, and each amino acid has a backbone of three atoms, N-C-C, which are always linked in the N to C direction: 

`Beginning of protein sequence amino acids:       1       2       3    ... etc`

`Beginning of protein sequence backbone atoms: (N-C-C)-(N-C-C)-(N-C-C)-... etc`

So in our final output, we need to have xyz coordinates for each of the three atoms, in order, for every amino acid.

## NGL - load a sample protein to get the parser working 

https://files.rcsb.org/download/5YHX.pdb

In [7]:
import nglview as nv
from Bio.PDB import PDBParser

# Load your PDB file
parser = PDBParser()
structure = parser.get_structure('Sample', './data/5yhx.pdb')

# Create NGLView widget for the structure
view = nv.show_biopython(structure)
view.clear_representations()

# Add molecular graphics: e.g., cartoon for secondary structures and ball+stick for atoms
view.add_representation('cartoon', selection='protein', color='blue')
view.add_representation('ball+stick', selection='ligand')

view

NGLWidget()

## Here is a parser to turn a row of our dataframe into a PDB file that can be viewed with NGL. Note that our dataset doesn't have info on the secondary structure, so these structures will not appear bolded in the output. The PDB specification is available here:

https://www.biostat.jhsph.edu/~iruczins/teaching/260.655/links/pdbformat.pdf

In [8]:
import pandas as pd

def format_pdb_from_df(row):
    pdb_str = ""
    atom_count = 1  # Starting index for ATOM records
    chain_id = 'A'  # Assuming a single chain for simplicity
    res_num = 1     # Residue number
    occupancy = 1.00
    t_factor = 50.00

    atom_types = ['N', 'CA', 'C']  # Backbone atoms
    element_types = {'N': 'N', 'CA': 'C', 'C': 'C'}  # Elements for atoms
    residue_mapping = {'A': 'ALA','R': 'ARG','N': 'ASN','D': 'ASP','C': 'CYS','E': 'GLU','Q': 'GLN','G': 'GLY',
                       'H': 'HIS','I': 'ILE','L': 'LEU','K': 'LYS','M': 'MET','F': 'PHE','P': 'PRO','S': 'SER',
                       'T': 'THR','W': 'TRP','Y': 'TYR','V': 'VAL' } # PDB requires the 3 letter codes for amino acids

    for i in range(len(row['Sequence'])):
        residue = residue_mapping[row['Sequence'][i]]
        # Loop over each backbone atom type
        for j, atom_type in enumerate(atom_types):
            # Index to pull the correct coordinates from flattened list
            idx = 3 * i + j
            x = row['x'][idx]/100
            y = row['y'][idx]/100
            z = row['z'][idx]/100
            element = element_types[atom_type]

            pdb_str += f"ATOM  {atom_count:>5}  {atom_type:<2}  {residue:>3} {chain_id}{res_num:>4}    {x:>8.3f}{y:>8.3f}{z:>8.3f}{occupancy:>6.2f}{t_factor:>6.2f}          {element:>2}  \n"
            atom_count += 1

        res_num += 1  # Increment residue number for each new amino acid

    return pdb_str

In [9]:
protein_row = processed_df.iloc[0]

pdb_content = format_pdb_from_df(protein_row)
print(pdb_content)

# Optionally, write to a file
with open("./data/sample_protein.pdb", "w") as file:
    file.write(pdb_content)

ATOM      1  N   ASP A   1       0.000   0.000   0.000  1.00 50.00           N  
ATOM      2  CA  ASP A   1       0.000   0.000   0.000  1.00 50.00           C  
ATOM      3  C   ASP A   1       0.000   0.000   0.000  1.00 50.00           C  
ATOM      4  N   ARG A   2       0.000   0.000   0.000  1.00 50.00           N  
ATOM      5  CA  ARG A   2       0.000   0.000   0.000  1.00 50.00           C  
ATOM      6  C   ARG A   2       0.000   0.000   0.000  1.00 50.00           C  
ATOM      7  N   VAL A   3       0.000   0.000   0.000  1.00 50.00           N  
ATOM      8  CA  VAL A   3       0.000   0.000   0.000  1.00 50.00           C  
ATOM      9  C   VAL A   3       0.000   0.000   0.000  1.00 50.00           C  
ATOM     10  N   TYR A   4       0.000   0.000   0.000  1.00 50.00           N  
ATOM     11  CA  TYR A   4       0.000   0.000   0.000  1.00 50.00           C  
ATOM     12  C   TYR A   4       0.000   0.000   0.000  1.00 50.00           C  
ATOM     13  N   ILE A   5  

In [10]:
import nglview as nv
from Bio.PDB import PDBParser

# Load your PDB file
parser = PDBParser()
structure = parser.get_structure('Sample', './data/sample_protein.pdb')

# Create NGLView widget for the structure
view = nv.show_biopython(structure)
view.clear_representations()

# Add molecular graphics: e.g., cartoon for secondary structures and ball+stick for atoms
view.add_representation('cartoon', selection='protein', color='blue')
view.add_representation('ball+stick', selection='ligand')

view

NGLWidget()

## add features for GNN
### For nodes (amino acids) we will add some of their chemical properties)

In [11]:
def amino_acid_properties():
    properties = {
        'Node_Hydrophobic': "AYCMVLWIF",
        'Node_Hydrophilic': "RKNEPD",
        'Node_Polar': "YWHKREQDNST",
        'Node_Small': "VCAGDNSTP",
        # 'Node_Proline': "P",
        'Node_Tiny': "AGS",
        'Node_Aliphatic': "ILV",
        'Node_Aromatic': "FYWH",
        'Node_Positive': "HKR",
        'Node_Negative': "DE",
        'Node_Charged': "HKRED"
    }
    return properties

def encode_properties(seq, properties):
    # Initialize the dictionary for property encodings
    property_encodings = {key: "" for key in properties}
    
    # Iterate through each amino acid in the sequence
    for aa in seq:
        for prop, aas in properties.items():
            if aa in aas:
                property_encodings[prop] += '1'
            else:
                property_encodings[prop] += '0'
    
    return property_encodings

In [12]:
df_features = processed_df.copy()

# Get amino acid properties
props = amino_acid_properties()

# Encode properties for each sequence
for prop in props:
    df_features[prop] = df_features['Sequence'].apply(lambda x: encode_properties(x, props)[prop])

df_features.rename(columns={#"Sequence": "Node_Sequence",
                            "Mask": "Node_Mask",
                            "ID": "Protein_Name",}, inplace=True)
df_features.head(1)

,Protein_Name,Sequence,Node_Mask,x,y,z,A,C,D,E,...,Node_Hydrophobic,Node_Hydrophilic,Node_Polar,Node_Small,Node_Tiny,Node_Aliphatic,Node_Aromatic,Node_Positive,Node_Negative,Node_Charged
0,90#2WXZ_2_C,DRVYIHPFHLLYYSKSTCAQLENPSVETLPEPTFEPVPIQAKTSPV...,0000011111111111111100000001111111111111111111...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,0011100101111000011010000100100001001010100001...,1100001000000010000001110010011100110100010010...,1101010010011111100101101011001010100001011100...,1010001000000101111000111101010110011100101111...,0000000000000101001000001000000000000000100100...,0010100001100000000010000100100000001010000001...,0001010110011000000000000000000001000000000000...,0100010010000010000000000000000000000000010000...,1000000000000000000001000010001000100000000000...,1100010010000010000001000010001000100000010000...


## Functions to compute global edge interactions (between all nodes)

In [13]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

def calculate_distances(coords):
    """Calculate the Euclidean distance matrix for a set of coordinates."""
    return squareform(pdist(coords, 'euclidean'))

def calculate_centroid(coordinates):
    """Calculate the centroid from a list of 3D coordinates."""
    return np.mean(coordinates, axis=0)

def calculate_residue_distance_matrix(centroid_list):
    """Calculate the Euclidean distance matrix for a list of centroids."""
    return squareform(pdist(centroid_list, 'euclidean'))

def calculate_sequence_distance(seq_length):
    """Calculate the sequence distance matrix for a protein sequence."""
    indices = np.arange(seq_length)
    seq_dist = np.abs(indices[:, None] - indices[None, :])
    return seq_dist

def calculate_contact_map(dist_matrix, threshold=16.0):
    """Calculate the contact map for a distance matrix given a threshold, ignoring distances of 0."""
    contact_map = (dist_matrix <= threshold).astype(int)
    contact_map[dist_matrix == 0] = 0
    return contact_map

## Functions to compute local edge interactions (between neighboring nodes)

In [14]:
def conservation_interaction_matrix(mask):
    """Generate a matrix where entries are 1 if both residues are conserved, 0 otherwise."""
    n = len(mask)
    conservation_matrix = np.zeros((n, n), dtype=int)
    for i in range(1, n):  # Start from 1 to be able to access i-1
        if mask[i] == '1' and mask[i-1] == '1':
            conservation_matrix[i, i-1] = 1  # Interaction with previous residue
        if i < n - 1 and mask[i] == '1' and mask[i+1] == '1':
            conservation_matrix[i, i+1] = 1  # Interaction with next residue
    return conservation_matrix

def calculate_charge_interactions(sequence):
    """Calculate electrostatic interaction matrix for a given amino acid sequence."""
    n = len(sequence)
    charge_matrix = np.zeros((n, n), dtype=int)
    charge_map = {'D': -1, 'E': -1, 'K': 1, 'R': 1, 'H': 1}  # Example simplified charges

    for i in range(1, n):
        if sequence[i] in charge_map and sequence[i-1] in charge_map:
            if charge_map[sequence[i]] * charge_map[sequence[i-1]] == -1:
                charge_matrix[i, i-1] = 1
        if i < n - 1 and sequence[i] in charge_map and sequence[i+1] in charge_map:
            if charge_map[sequence[i]] * charge_map[sequence[i+1]] == -1:
                charge_matrix[i, i+1] = 1

    return charge_matrix

import numpy as np

def assign_interaction_group(aa):
    """Assign interaction group to amino acids."""
    hydrophobic = {'L', 'I', 'M', 'P', 'V', 'A', 'G', 'F', 'W'}
    hydrophilic = {'D', 'E', 'H', 'K', 'R', 'Q', 'N', 'C', 'T', 'S'}
    if aa in hydrophobic: return -1  # Hydrophobic
    elif aa in hydrophilic: return 1  # Hydrophilic
    return 0  # Neither

def calculate_water_interactions(sequence):
    """Calculate the interaction matrix for immediate neighbors in the sequence."""
    n = len(sequence)
    interaction_scores = np.array([assign_interaction_group(aa) for aa in sequence])
    
    # Create an empty matrix with zeros
    interaction_matrix = np.zeros((n, n), dtype=int)

    # Only set values for immediate neighbors
    for i in range(1, n):
        if interaction_scores[i] == interaction_scores[i-1] and interaction_scores[i] != 0:
            interaction_matrix[i, i-1] = interaction_scores[i]
            interaction_matrix[i-1, i] = interaction_scores[i]  # Ensure symmetry

    return interaction_matrix

## apply all functions

In [15]:
# rescale distances to Angstroms
df_features['x'] = df_features['x'].apply(lambda x: [val / 100 for val in x])
df_features['y'] = df_features['y'].apply(lambda x: [val / 100 for val in x])
df_features['z'] = df_features['z'].apply(lambda x: [val / 100 for val in x])
df_features['Coordinates'] = df_features.apply(lambda row: list(zip(row['x'], row['y'], row['z'])), axis=1)
    
# Calculate atom distance matrix and add to dataframe
tqdm.pandas(desc="Calculating 3D atom distance matrices")
df_features['Atom_3D_dists'] = df_features['Coordinates']\
                                 .progress_apply(lambda coords: calculate_distances(np.array(coords)))

# Calculate centroid for each residue
tqdm.pandas(desc="Calculating residue centroids")
df_features['Node_centroids'] = df_features['Coordinates']\
        .progress_apply(lambda coords: [calculate_centroid(coords[i:i+3]) for i in range(0, len(coords), 3)])
    
# Calculate distance matrix between centroids
tqdm.pandas(desc="Calculating residue 3D distances")
df_features['Edge_3D_dists_global'] = df_features['Node_centroids']\
        .progress_apply(lambda centroids: calculate_residue_distance_matrix(np.array(centroids)))

# Calculate sequence distance matrix and add to dataframe
tqdm.pandas(desc="Calculating sequence distance matrices")
df_features['Edge_1D_dists_global'] = df_features['Sequence']\
                                    .progress_apply(lambda seq: calculate_sequence_distance(len(seq)))

# Calculate which other atoms are within 16A, which is close enough to make a secondary structure
tqdm.pandas(desc="Calculating atom contact map matrices")
df_features['Atom_contact'] = df_features['Atom_3D_dists']\
                                  .progress_apply(lambda dist_matrix: calculate_contact_map(dist_matrix))

# Calculate which other residues are within 16A, which is close enough to make a secondary structure
tqdm.pandas(desc="Calculating residue contact map matrices")
df_features['Edge_proximity_global'] = df_features['Edge_3D_dists_global']\
        .progress_apply(lambda dist_matrix: calculate_contact_map(dist_matrix))

# Calcluate which edges are highly conserved
tqdm.pandas(desc="Calculating conservation matrices")
df_features['Edge_conserved_local'] = df_features['Node_Mask']\
                                .progress_apply(lambda mask: conservation_interaction_matrix(mask))

# Calcluate charge interactions
tqdm.pandas(desc="Calculating charge matrices")
df_features['Edge_charge_local'] = df_features['Sequence']\
                                .progress_apply(lambda seq: calculate_charge_interactions(seq))

# Calcluate hydrophobicity interactions
tqdm.pandas(desc="Calculating solubility matrices")
df_features['Edge_solubility_local'] = df_features['Sequence']\
                                .progress_apply(lambda seq: calculate_water_interactions(seq))

Calculating residue 3D distances: 100%|██████| 224/224 [00:00<00:00, 8591.58it/s]
Calculating sequence distance matrices: 100%|█| 224/224 [00:00<00:00, 12465.33it/
Calculating atom contact map matrices: 100%|█| 224/224 [00:00<00:00, 1430.78it/s]
Calculating residue contact map matrices: 100%|█| 224/224 [00:00<00:00, 12548.91i
Calculating solubility matrices: 100%|███████| 224/224 [00:00<00:00, 7766.20it/s]


### convert strings to arrays and map amino acids to numbers

In [16]:
def convert_strings_to_char_arrays(df, columns):
    aa_to_int = {'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9,
                 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17,
                 'V': 18, 'W': 19, 'Y': 20}
    # Apply conversion to the 'Sequence' column specifically
    df['Sequence'] = df['Sequence'].apply(lambda seq: [aa_to_int[aa] for aa in seq])
    
    # Define a vectorized function to split strings into arrays of characters
    vectorized_split = np.vectorize(list)
    # Apply the vectorized function to the specified columns
    for column in columns:
        df[column] = df[column].apply(vectorized_split)
    return df
    
# Specify the columns you want to convert
columns_to_convert = ['Node_Mask', 'Node_Hydrophobic', 'Node_Hydrophilic',
                      'Node_Polar', 'Node_Small', 'Node_Tiny', 'Node_Aliphatic',          
                      'Node_Aromatic', 'Node_Positive', 'Node_Negative', 'Node_Charged']          

# Convert the specified columns
df_features = convert_strings_to_char_arrays(df_features, columns_to_convert)

## Current features

In [17]:
df_features.iloc[0].T

Protein_Name                                                   90#2WXZ_2_C
Sequence                 [3, 15, 18, 20, 8, 7, 13, 5, 7, 10, 10, 20, 20...
Node_Mask                [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
x                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
y                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
z                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
A                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012...
C                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002...
D                        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
E                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
F                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.016...
G                        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
H                        [0.0, 0.0, 0.0, 0.021224489795918365, 0.0, 1.0...
I                        

In [18]:
df_features.columns

Index(['Protein_Name', 'Sequence', 'Node_Mask', 'x', 'y', 'z', 'A', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T',
       'V', 'W', 'Y', 'Info', 'Node_Hydrophobic', 'Node_Hydrophilic',
       'Node_Polar', 'Node_Small', 'Node_Tiny', 'Node_Aliphatic',
       'Node_Aromatic', 'Node_Positive', 'Node_Negative', 'Node_Charged',
       'Coordinates', 'Atom_3D_dists', 'Node_centroids',
       'Edge_3D_dists_global', 'Edge_1D_dists_global', 'Atom_contact',
       'Edge_proximity_global', 'Edge_conserved_local', 'Edge_charge_local',
       'Edge_solubility_local'],
      dtype='object')

## model

In [19]:
# !pip install torch torch-geometric ogb

In [19]:
import torch
from torch_geometric.data import Data, DataLoader
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [20]:
df = df_features.copy(deep=True)

In [25]:
def df_to_graph(row):
    # node features (assuming they are already properly scaled and do not include the raw sequence anymore)
    features = np.vstack([ row['Sequence'],
        row['A'], row['C'], row['D'], row['E'], row['F'],
        row['G'], row['H'], row['I'], row['K'], row['L'],
        row['M'], row['N'], row['P'], row['Q'], row['R'],
        row['S'], row['T'], row['V'], row['W'], row['Y'],
        row['Node_Hydrophobic'], row['Node_Hydrophilic'],
        row['Node_Polar'], row['Node_Small'], row['Node_Tiny'],
        row['Node_Aliphatic'], row['Node_Aromatic'], row['Node_Positive'],
        row['Node_Negative'], row['Node_Charged'], row['Info'],
    ]).astype(float).T  # Ensure all data is float

    # Combine sequence encoding with additional features
    node_features = torch.tensor(features, dtype=torch.float)

    # Edge indices for consecutive residues
    edge_index = torch.tensor(np.vstack([np.arange(len(row['Sequence']) - 1), np.arange(1, len(row['Sequence']))]), dtype=torch.long)

    # Edge attributes
    edge_attributes = torch.tensor(np.vstack([
        row['Edge_1D_dists_global'],
        row['Edge_conserved_local'],
        row['Edge_charge_local'],
        row['Edge_solubility_local']
    ]).T, dtype=torch.float)  # Transpose to align features with edges

    # Target (residue centroids)
    target = torch.tensor(row['Node_centroids'], dtype=torch.float)

    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=target)

# Apply this function to a DataFrame row by row
# graphs = df.apply(df_to_graph, axis=1)  # This might still cause an error due to inconsistent feature lengths across rows.

In [26]:
# Example of applying this function to your DataFrame
graphs = df.apply(df_to_graph, axis=1)

NameError: name 'edge_attr' is not defined

In [ ]:
df.iloc[20]

### plotting functions from features

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_distance_distribution_specific_atoms(dist_matrix, start_index=630, end_index=666):
#     """
#     Plot the distribution of distances between a specific range of atoms in a protein.
    
#     Args:
#     - dist_matrix (numpy.ndarray): The distance matrix of a protein.
#     - start_index (int): Start index of the atom range.
#     - end_index (int): End index of the atom range (inclusive).
#     """
#     # Extract the specific submatrix for the desired range
#     sub_matrix = dist_matrix[start_index:end_index+1, start_index:end_index+1]
    
#     # Flatten the upper triangle of the submatrix, ignoring diagonal (distance 0)
#     distances = sub_matrix[np.triu_indices_from(sub_matrix, k=1)]
    
#     # Plot the distribution of distances
#     plt.figure(figsize=(10, 6))
#     sns.histplot(distances, bins=50, kde=True)
#     plt.title(f'Distribution of Distances Between Atoms {start_index} to {end_index}')
#     plt.xlabel('Distance (Å)')
#     plt.ylabel('Frequency')
#     plt.grid(True)
#     plt.show()

# # Example usage:
# # Assuming 'dist_matrix' is already defined and contains the distances for all atoms
# plot_distance_distribution_specific_atoms(dist_matrix)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_distance_distribution_specific_atoms(dist_matrix, start_index=0, end_index=5000):
#     """
#     Plot the distribution of distances between a specific range of atoms in a protein.
    
#     Args:
#     - dist_matrix (numpy.ndarray): The distance matrix of a protein.
#     - start_index (int): Start index of the atom range.
#     - end_index (int): End index of the atom range (inclusive).
#     """
#     # Extract the specific submatrix for the desired range
#     sub_matrix = dist_matrix[start_index:end_index+1, start_index:end_index+1]
    
#     # Flatten the upper triangle of the submatrix, ignoring diagonal (distance 0)
#     distances = sub_matrix[np.triu_indices_from(sub_matrix, k=1)]
    
#     # Plot the distribution of distances
#     plt.figure(figsize=(4, 3))
#     sns.histplot(distances, bins=2,)
#     plt.title(f'Distribution of Distances Between Atoms {start_index} to {end_index}')
#     plt.xlabel('Distance (Å)')
#     plt.ylabel('Frequency')
#     plt.grid(True)
#     plt.show()

# # Example usage:
# # Assuming 'dist_matrix' is already defined and contains the distances for all atoms
# plot_distance_distribution_specific_atoms(df_features['Edge_contact_map'].iloc[0])


In [ ]:
# # contact matrix visualizer
# import matplotlib.pyplot as plt
# import numpy as np

# def plot_section_of_array(array, start_row, start_col, max):
#     """
#     Plot a 20x20 section of a numpy 2D array starting from specified row and column indices.
    
#     Args:
#     - array (numpy.ndarray): The 2D array to visualize.
#     - start_row (int): The starting row index for the 20x20 section.
#     - start_col (int): The starting column index for the 20x20 section.
#     """
#     if array.shape[0] < start_row + max or array.shape[1] < start_col + max:
#         raise ValueError("Array does not contain enough rows or columns for a 20x20 section from the starting index.")
    
#     # Extract the 20x20 section from the array
#     section = array[start_row:start_row + max, start_col:start_col + max]
    
#     # Plot the section using imshow
#     plt.figure(figsize=(6, 6))
#     plt.imshow(section, cmap='viridis', interpolation='nearest')
#     plt.colorbar()
#     plt.title('20x20 Section of Array')
#     plt.show()

# # Example usage:
# # Assume 'large_array' is a numpy 2D array larger than 20x20
# mat = df_features['Edge_residue_contact'].iloc[0]
# plot_section_of_array(mat, 0, 0, len(mat))  # Adjust indices based on your array dimensions

In [ ]:
# import pandas as pd

# def format_pdb_from_df1(row):
#     pdb_str = ""
#     atom_count = 1  # Starting index for ATOM records
#     chain_id = 'A'  # Assuming a single chain for simplicity
#     res_num = 1     # Residue number
#     occupancy = 1.00
#     t_factor = 50.00

#     # Only keep the alpha carbon (CA)
#     atom_types = ['CA']  # Only backbone alpha carbon atoms
#     element_types = {'CA': 'C'}  # Element for alpha carbon
    
#     residue_mapping = {'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS', 
#                        'E': 'GLU', 'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 
#                        'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO', 
#                        'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'}

#     for i in range(len(row['Sequence'])):
#         residue = residue_mapping[row['Sequence'][i]]
#         # We know the alpha carbon is the second atom in the list for each residue (1,4,7,...)
#         idx = 3 * i + 1  # Update index to only point to alpha carbon
#         x = row['x'][idx]/100
#         y = row['y'][idx]/100
#         z = row['z'][idx]/100
#         element = element_types['CA']

#         pdb_str += f"ATOM  {atom_count:>5}  {atom_types[0]:<2}  {residue:>3} {chain_id}{res_num:>4}    {x:>8.3f}{y:>8.3f}{z:>8.3f}{occupancy:>6.2f}{t_factor:>6.2f}          {element:>2}  \n"
#         atom_count += 1
#         res_num += 1  # Increment residue number for each new amino acid

#     return pdb_str

# test = format_pdb_from_df1(protein_row)